In [1]:
import os
os.chdir(r"C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification")
print("Current directory:", os.getcwd())

Current directory: C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification


In [4]:
import dagshub
dagshub.init(repo_owner='bavanreddy1999', repo_name='End-to-End-Chicken-Disease-Classification', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\user\Python Programs\Resume 
Projects\End-to-End-Chicken-Disease-Classification\cnnProject\lib\site-packages\rich\live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d2982dba-ab02-493d-b2e0-c094015ce8c3&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ee6c2511ce81382f92ece07bc83df56ba3415bf897470b486dcca29116920219




Accessing as bavanreddy1999

Initialized MLflow to track repo "bavanreddy1999/End-to-End-Chicken-Disease-Classification"

Repository bavanreddy1999/End-to-End-Chicken-Disease-Classification initialized!

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model:Path
    training_data:Path
    all_params:dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnProject.utils.common import readYamlFile,createDirectories

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=readYamlFile(config_file_path)
        self.params=readYamlFile(params_file_path)
        createDirectories([self.config.artifacts_root])

    def evaluation_config(self)-> EvaluationConfig:
        eval_config=EvaluationConfig(
            path_to_model='artifacts/training/model.h5',
            training_data='artifacts/data_ingestion/Chicken-fecal-images',
            mlflow_uri="https://dagshub.com/bavanreddy1999/End-to-End-Chicken-Disease-Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
            )
        return eval_config

In [ ]:
import tensorflow as tf
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from cnnProject.utils.common import saveJSON

In [11]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [16]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config

    def _valid_generator(self):
        datagenerators_kwargs=dict(rescale=1./255,validation_split=0.30)
        dataflow_kwargs=dict(target_size=self.config.params_image_size[:-1],
                             batch_size=self.config.params_batch_size,
                             interpolation="bilinear")
        
        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(**datagenerators_kwargs)

        self.valid_generator=valid_datagenerator.flow_from_directory(directory=self.config.training_data,subset="validation",shuffle=False,**dataflow_kwargs)

    @staticmethod
    def load_model(path:Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        saveJSON(path=Path("scores.json"),data=scores)

    def log_into_mlfow(self):
        mlflow.set_registry_uri(uri=self.config.mlflow_uri)
        tracking_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss":self.score[0],"accuracy":self.score[1]})

            if tracking_uri_type_store!="file":
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")

In [17]:
try:
    config=ConfigurationManager()
    eval_config=config.evaluation_config()
    evaluation=Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlfow()
except Exception as e:
    raise e

[2025-06-29 18:49:22,808 : INFO : common : the yaml file form the path config\config.yaml loaded Sucessfully]
[2025-06-29 18:49:22,809 : INFO : common : the yaml file form the path params.yaml loaded Sucessfully]
[2025-06-29 18:49:22,810 : INFO : common : Directories created Sucessfully at artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 4s 438ms/step - loss: 0.3358 - accuracy: 0.9741


2025/06/29 18:49:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-06-29 18:49:28,463 : WARNING : save : Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2025-06-29 18:49:28,845 : INFO : builder_impl : Assets written to: C:\Users\user\AppData\Local\Temp\tmp5gxwr2u4\model\data\model\assets]


Successfully registered model 'VGG16Model'.
2025/06/29 18:50:03 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
